In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Particulas de la Barra en z = 0 - M31gd

In [18]:
import h5py
import rotation as rot
import barstrength2 as strng
import time_conversion as tiempo
import tenform as ten
from numpy import linalg


#Parámetros cosmológicos
#--------------------------------------------------------------------------
h=0.732
G = 4.299e-6
a0=1.
H0 = h*100
omega_lambda=0.716
omega_matter=0.1277/(h**2.)
omega0 = omega_lambda + omega_matter
#---------------------------------------------------------------------------
aa = np.loadtxt('/home/ornela/SimCLUES/redshift_outputs.txt')
aexp = aa[:,2]

data = np.loadtxt('/home/ornela/PYTHON/Barras_GdGs/Barras_Gd/_data/M31_lbarvstime.dat')
lbar1 = data[:,1]
lbar2 = data[:,2]

snapshot=range(496,495,-1) #SNAPSHOTS
i=0
for isnap in snapshot:
    
    snap = h5py.File('/home/ornela/SimCLUES/outputs_1/snap_'+str('%03d'%isnap)+'.h5py', 'r')

    cm    = snap['subhalo_000/Center'].value
    r200  = snap['subhalo_000/R200'].value
    IDs   = snap['subhalo_000/Str/Ids'].value
    formt = snap['subhalo_000/Str/FormationTime'].value
    pstr  = snap['subhalo_000/Str/Coordinates'].value
    mstr  = snap['subhalo_000/Str/Masses'].value
    vel   = snap['subhalo_000/Str/Velocities'].value
  
    z     = a0/aexp[isnap] - 1.
    Ht    = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)
    time  = tiempo.conv(z, h, omega_lambda, omega_matter)
    
    aux   = a0/formt - 1.
    tform = tiempo.conv(aux, h, omega_lambda, omega_matter)

#paso las coordenadas al centro de la galaxia y paso a coordenadas físicas
    xstr = (pstr[:,0]-cm[0])*aexp[isnap]/h
    ystr = (pstr[:,1]-cm[1])*aexp[isnap]/h
    zstr = (pstr[:,2]-cm[2])*aexp[isnap]/h
    r    = np.sqrt(xstr**2+ystr**2+zstr**2)
    
#paso las velocidades a unidades físicas
    v_x = vel[:,0] *np.sqrt(aexp[isnap]) + Ht *xstr/1000.
    v_y = vel[:,1] *np.sqrt(aexp[isnap]) + Ht *ystr/1000.
    v_z = vel[:,2] *np.sqrt(aexp[isnap]) + Ht *zstr/1000.


    mstr = mstr/h     #masa a unidades físicas

    r200 = r200*aexp[isnap]/h #radio virial a unidades fisicas

    rgal = 0.15*r200          #defino el radio de la galaxia

    limit = np.where(r<rgal)         #hago un límite de los r dentro de rgal
    r_sort = np.sort(r[limit])       #acomodo los r de menor a mayor
    r_indice = np.argsort(r[limit])  #saco los argumentos de r acomodados de menor a mayor

    Mc_str = cumsum(mstr[limit][r_indice])  #masa acumulada en cada radio dentro de rgal
    Mgal = Mc_str[-1]  #masa total de la galaxia

#Calculo de los radios a 50% y 90% la masa de la galaxia
#-----------------------------------------------------------------------
    razon = Mc_str/Mgal 
    noventa, = np.where(razon < 0.9)
    cincuenta, = np.where(razon < 0.5)

    r90 = r_sort[noventa][-1]   #radio a 90% la masa
    r50 = r_sort[cincuenta][-1] #radio a 50% la masa
    
#-----------------------------------------------------------------------
#componentes de la velocidad del centro de masa
    veloc, = np.where(r < r50)
    
    vxcm = sum(mstr[veloc]*v_x[veloc])/sum(mstr[veloc])
    vycm = sum(mstr[veloc]*v_y[veloc])/sum(mstr[veloc])
    vzcm = sum(mstr[veloc]*v_z[veloc])/sum(mstr[veloc])

#velocidades de las estrellas respecto del centro de masa de la galaxia
    vx = v_x - vxcm
    vy = v_y - vycm
    vz = v_z - vzcm

#hago la rotación de las coordenadas respecto del momento angular    
    e1x,e2x,e3x,e1y,e2y,e3y,e1z,e2z,e3z = rot.rot1(mstr,xstr,ystr,zstr,vx,vy,vz,3*aexp[isnap])

    xn = e1x*xstr + e1y*ystr + e1z*zstr
    yn = e2x*xstr + e2y*ystr + e2z*zstr
    zn = e3x*xstr + e3y*ystr + e3z*zstr
    vxn = e1x*vx + e1y*vy + e1z*vz
    vyn = e2x*vx + e2y*vy + e2z*vz
    vzn = e3x*vx + e3y*vy + e3z*vz
    
    rn  = np.sqrt(xn**2 + yn**2 + zn**2)
    
    #--------------------------------------------------------------------------------------    

    corte, = np.where(r<3*aexp[isnap])
    
    A2max, rmax, phimax = strng.a2max(mstr[corte], xn[corte], yn[corte], 20)
    
#     print phi, lbar2[i]
    
    xx = xn*cos(phimax)+yn*sin(phimax)
    yy = xn*-sin(phimax)+yn*cos(phimax)
    zz = zn
    
    #----para lbar2--------------------------------------------------------------
    limit1,=np.where(r < lbar2[i])
    tensor = ten.tenf(xx[limit1], yy[limit1], zz[limit1])

    matriz = linalg.eig(tensor)
    autov  = matriz[0]

    asort  = np.sort(autov)

    a1 = asort[2]
    b1 = asort[1]
    c1 = asort[0]
    
    A1 = np.sqrt(a1)
    B1 = np.sqrt(b1)
    C1 = np.sqrt(c1)

    
#Definimos el tamaño de nuestro elipsoide
    aa=lbar2[i]
    bb=(B1/A1)*lbar2[i]
    cc=(C1/A1)*lbar2[i]
    
#Hacemos la rotación de las coordenadas

    RR = np.sqrt((xx/aa)**2+(yy/bb)**2+(zz/cc)**2) # formula del elipsoide
    
    mask, = np.where(RR<1) #seleccionamos las particulas dentro del elipsoide
    
    ID    = IDs[mask]
    tform = tform[mask]
    rn_z0 = rn[mask]
    
    print len(ID)
    
#Guargamos los parámetros en distintos archivos
#----------------------------------------------------------------------------------
#     datos = np.ndarray([len(ID),3])
#     datos[:,0] = ID
#     datos[:,1] = tform
#     datos[:,2] = rn_z0

#     np.savetxt('/home/ornela/PYTHON/Barras_GdGs/Barras_Gd/_data/part_barra.dat', datos, fmt=('%15d' '%12.6f' '%12.6f'))
    
#----------------------------------------------------------------------------------  
    
#     fig=plt.figure(1, figsize=(8,8))
#     fig.subplots_adjust(bottom=0.12, left =0.16, right = 0.95, top = 0.95)
#     ax=fig.add_subplot(111)
# #     ax.plot(xn,yn,'.', markersize=0.1, color='k')
#     ax.plot(xn[mask],yn[mask],'.', markersize=0.1, color='r')
# #     ax.set_xlim(-5,5)
# #     ax.set_ylim(-5,5)
#     ax.minorticks_on()
#     ax.tick_params( labelsize=22)
#     ax.tick_params('both', length=3, width=1.2,which='minor', direction='in', right='on',top='on')
#     ax.tick_params('both', length=6, width=1.2,which='major', direction='in', right='on',top='on')  
#     ax.set_xlabel(r'$x\:[kpc]$', fontsize=26)
#     ax.set_ylabel(r'$y\:[kpc]$',fontsize=26)
#     plt.show()

186584


## Calculo el centro de masa para todos los tiempos

In [17]:
import time_conversion as tiempos

#Parámetros cosmológicos
#--------------------------------------------------------------------------
h=0.732
G = 4.299e-6
a0=1.
H0 = h*100
omega_lambda=0.716
omega_matter=0.1277/(h**2.)
omega0 = omega_lambda + omega_matter
#---------------------------------------------------------------------------
aa = np.loadtxt('/home/ornela/SimCLUES/redshift_outputs.txt')
aexp = aa[:,2]

snapshot=range(496,0,-1) #SNAPSHOTS
i=0
for isnap in snapshot:
    
    snap = h5py.File('/home/ornela/SimCLUES/outputs_1/snap_'+str('%03d'%isnap)+'.h5py', 'r')

    cm    = snap['subhalo_000/Center'].value
    
    z     = a0/aexp[isnap] - 1.
    Ht    = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)
    time  = tiempo.conv(z, h, omega_lambda, omega_matter)
    
    archivo1 =  open('/home/ornela/PYTHON/Barras_GdGs/Barras_Gd/_data/M31_masscenter_time.dat','a')
    archivo1.write(str('%12.12f'% time) +'\t'+
               str('%12.12f'% cm[0]) +'\t'+
               str('%12.12f'% cm[1]) +'\t'+
               str('%12.12f'% cm[2]) +'\n')
    archivo1.close()
    
#     pstr  = snap['subhalo_000/Str/Coordinates'].value